# RAG (Retrieval Augmented Generation)
- RAG is a technique using or retrieving data from private or real-time sources to increase, expand the capabilities of the LLMs.
![image](./images/Architecture.png)

## Retrieval

![image](./images/data_connection-95ff2033a8faa5f3ba41376c0f6dd32a.jpg)

1. From the source, load the data.
2. Transform it by splitting the data.
3. Embed the data.
4. Store the number (data).
5. Perform a number search.

- Document Loaders
    - Loader is a piece of code extract the data from a source and brings it to Langchain.

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader

# loader = TextLoader("./files/chapter_one.txt")
loader = PyPDFLoader("./files/chapter_one.pdf")

loader.load()

[Document(page_content="Mr.\nJones,\nof\nthe\nManor\nFarm,\nhad\nlocked\nthe\nhen-houses\nfor\nthe\nnight,\nbut\nwas\ntoo\ndrunk\nto\nremember\nto\nshut\nthe\npop-holes.\nWith\nthe\nring\nof\nlight\nfrom\nhis\nlantern\ndancing\nfrom\nside\nto\nside,\nhe\nlurched\nacross\nthe\nyard,\nkicked\noff\nhis\nboots\nat\nthe\nback\ndoor,\ndrew\nhimself\na\nlast\nglass\nof\nbeer\nfrom\nthe\nbarrel\nin\nthe\nscullery,\nand\nmade\nhis\nway\nup\nto\nbed,\nwhere\nMrs.\nJones\nwas\nalready\nsnoring.\nAs\nsoon\nas\nthe\nlight\nin\nthe\nbedroom\nwent\nout\nthere\nwas\na\nstirring\nand\na\nfluttering\nall\nthrough\nthe\nfarm\nbuildings.\nWord\nhad\ngone\nround\nduring\nthe\nday\nthat\nold\nMajor,\nthe\nprize\nMiddle\nWhite\nboar,\nhad\nhad\na\nstrange\ndream\non\nthe\nprevious\nnight\nand\nwished\nto\ncommunicate\nit\nto\nthe\nother\nanimals.\nIt\nhad\nbeen\nagreed\nthat\nthey\nshould\nall\nmeet\nin\nthe\nbig\nbarn\nas\nsoon\nas\nMr.\nJones\nwas\nsafely\nout\nof\nthe\nway.\nOld\nMajor\n(so\nhe\nwas\nalwa